In [0]:
#get the file name from the adf
fileName='Products.csv'
#fileName = dbutils.widgets.get('fileName')
fileNameWithoutExt = fileName.split('.')[0]
print(fileNameWithoutExt)

All the sensitive information has been stored in the key vault. We established the connection with the key vault from the databricks secret scope. And feteching the information using the databricks secrets get method.

In [0]:
import pyspark.sql.functions as F
#from datetime import datetme as dt

#Just change all the values here based on the resource name you have created in your environemnt and workspace.
#All the key values are assinged to the variables.

sqlDbName = 'abdproject '
dbUserName = 'abduser'
passwordKey = 'adbuserkey'
stgAccountSASTokenKey = 'abhinay250898SasToken'
landingFileName =fileName #'Product'  #dbutils.widgets.get('Product')
databricksScopeName ='ApMorganSecret'
dbServer = 'abdserver'
dbServerPortNumber ='1433'
storageContainer ='apmorgan'
storageAccount='abhinay250898adls'
landingMountPoint ='/mnt'


In [0]:
dbutils.secrets.get(scope = 'KeyVaultAccess', key= stgAccountSASTokenKey) #KeyVaultAccess

In [0]:

client_id = dbutils.secrets.get(scope=databricksScopeName, key="ClientId")       # Service Principal App (Client) ID
client_secret = dbutils.secrets.get(scope=databricksScopeName, key="ClientSecret")  # Client Secret
tenant_id = dbutils.secrets.get(scope=databricksScopeName, key="TenantId")      # Tenant ID

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}


In [0]:
dbutils.fs.mount(source="abfss//container@storageaccount.dfs.core.window.net")

In [0]:
if not any(mount.mountPoint == landingMountPoint for mount in dbutils.fs.mounts()):
    dbutils.fs.mount(
            source = f"abfss://{storageContainer}@{storageAccount}.dfs.core.windows.net/",
            mount_point = "/mnt",
            extra_configs = configs)
    print('Storage account created')
else:
    print('Storage account already mounted')

In [0]:
#connect to Azure SQL DB
dbPassword = dbutils.secrets.get(scope = databricksScopeName, key= passwordKey)
serverurl = 'jdbc:sqlserver://{}.database.windows.net:{};database={};user={};'.format(dbServer, dbServerPortNumber,sqlDbName,dbUserName)
connectionProperties = {
    'password':dbPassword,
    'driver':'com.microsoft.sqlserver.jdbc.SQLServerDriver'
}
df = spark.read.jdbc(url = serverurl, table = 'dbo.FileDetailsFormat', properties= connectionProperties)
display(df)


In [0]:
df1 = spark.read.csv('/mnt/landing/'+fileName, inferSchema=True, header=True)
#display(df1)

# Rule
errorFlag=False
errorMessage = ''
totalcount = df1.count()
print(totalcount)
distinctCount = df1.distinct().count()
print(distinctCount)
if distinctCount !=totalcount:
    errorFlag = True
    errorMessage = 'Duplication Found. Rule 1 Failed'
print(errorMessage)
    
# Rule 2
df2 = df.filter(df.FileName==fileNameWithoutExt).select('ColumnName','ColumnDateFormat' )
rows = df2.collect()
for r in rows:
    colName = r[0]
    colFormat =r[1]
    print(colName, colFormat)
    #display(df1.filter(F.to_date(colName, colFormat).isNull() ==True))
    formatCount =df1.filter(F.to_date(colName, colFormat).isNotNull() ==True).count()
    if formatCount == totalcount:
        errorFlag = True
        errorMessage = errorMessage +' DateFormate is incorrect for {} '.format(colName)
    else:
        print('All rows are good for ', colName)
print(errorMessage)

if errorFlag:
    dbutils.fs.mv('/mnt/landing/'+fileName,'/mnt/rejected/'+fileName )
    dbutils.notebook.exit('{"errorFlag": "true", "errorMessage":"'+errorMessage +'"}')
else:
    dbutils.fs.mv('/mnt/landing/'+fileName,'/mnt/staging/'+fileName )
    dbutils.notebook.exit('{"errorFlag": "false", "errorMessage":"No error"}')